# Emulator
I am using Gaussian Process to build an Emulator of FOM(Figure Of Merit) in 6-D parameter Space. I have predicted a FOM value for last raw from given data. Results are shown as the value of predicted FOM and given FOM.

In [1]:
import george
import numpy as np
import matplotlib.pyplot as pl
from george import kernels
from scipy.optimize import minimize
from george.metrics import Metric
%matplotlib inline

In [2]:
a = np.loadtxt("parameters_with_FOM.txt")

In [3]:
area = a[:,0]
depth = a[:,1]
shear_m = a[:,2]
sigma_z = a[:,3]
sig_delta_z = a[:,4]
sig_sigma_z= a[:,5]
FOM = a[:,6]
#slicing up the array in 7 column array

In [4]:
Sarea = (area-np.mean(area))/np.std(area)
Sdepth = (depth-np.mean(depth))/np.std(depth)
Sshear_m = (shear_m-np.mean(shear_m))/np.std(shear_m)
Ssigma_z = (sigma_z-np.mean(sigma_z))/np.std(sigma_z)
Ssig_delta_z = (sig_delta_z-np.mean(sig_delta_z))/np.std(sig_delta_z)
Ssig_sigma_z = (sig_sigma_z-np.mean(sig_sigma_z))/np.std(sig_sigma_z)
SFOM = (FOM-np.mean(FOM))/np.std(FOM)
#standardizing the data

In [5]:
x = np.column_stack([Sarea, Sdepth, Sshear_m, Ssigma_z, Ssig_delta_z, Ssig_sigma_z])
#Putting together standardised parameter array

In [6]:
kernel = kernels.ExpSquaredKernel(metric=[1,1,1,1,1,1], ndim=6)
#creating a kernel-covariance in 6-D parameter space

In [7]:
gp = george.GP(kernel, mean=np.mean(SFOM))

In [8]:
gp.compute(x[:-1])

def neg_ln_lik(p):
            gp.set_parameter_vector(p)
            return -gp.log_likelihood(SFOM[:-1]) 
def grad_neg_ln_like(p):
            gp.set_parameter_vector(p)
            return -gp.grad_log_likelihood(SFOM[:-1]) 

result = minimize(neg_ln_lik, gp.get_parameter_vector(), jac=grad_neg_ln_like)

gp.set_parameter_vector(result.x)
#running Gaussian process

In [9]:
PSFOM, PerameterValues = gp.predict(SFOM[:-1], [x[-1]])
#Predicting the test point(Which is the last raw of data here)

In [10]:
PFOM = (PSFOM*np.std(FOM))+np.mean(FOM)
#De-standardizing the standardised predicted FOM value

In [11]:
PFOM[0], FOM[-1]
#Comparing the predicted FOM and given FOM

(13265.214697708, 13265.22)